Multi-label image classification

https://www.analyticsvidhya.com/blog/2019/04/build-first-multi-label-image-classification-model-python/

Use dl1-lab env

In [0]:
import keras

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping, History, CSVLogger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import pickle


In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
import os
os.listdir()
from google.colab import  drive
drive.mount('/content/drive')

In [0]:
proj_dir = '/content/drive/My Drive/Colab Notebooks/analytics-vidhya-multi-label-images/'
os.listdir(proj_dir)
data_dir = '/content/drive/My Drive/Colab Notebooks/data/multi-label-images/'
os.listdir(data_dir)

In [0]:
#read csv file
csvPath =  proj_dir + 'multi-label-movie-genre.csv'

In [0]:
CSVLogFile = proj_dir + 'CSVLogFile_Training.csv'

In [0]:
print(CSVLogFile)

In [0]:
trainingHistory = proj_dir + 'resultsTrainingHistory.pkl'

In [0]:
data = pd.read_csv(csvPath)

In [0]:
data.head()

In [0]:
data.shape

In [0]:
data.describe()

In [0]:
data.info()

In [0]:
data.columns

In [0]:
#macbook
#curDir = os.path.abspath('.')
#imagePath = os.path.sep.join([curDir, 'data/multi-label-images/'])
#print(imagePath)

In [0]:
imagePath=data_dir + 'multi-label-images/'
print(imagePath)

In [0]:
data_image=[]
for i in tqdm(range(data.shape[0])):
    img=image.load_img(imagePath + data['Id'][i]+'.jpg', target_size=(300,300,3))
    img=image.img_to_array(img)
    img=img/255.
    data_image.append(img)
    
 

In [0]:
len(data_image)

In [0]:
X=np.array(data_image)   

In [0]:
print(X.shape)

In [0]:
np.save(proj_dir + 'X_array.dat', X)

In [0]:
#show 1 image
plt.imshow(X[2])
plt.text(2,2,data['Genre'][2])

In [0]:
#get target labels array and drop Id and Genre
y=np.array(data.drop(['Id','Genre'],axis=1))
print(y.shape)
print('example - comedy is 5th column -  ',y[2])

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y, random_state=42, test_size=0.2)

In [0]:
#with open('X_train.csv',  'w') as FOUT:
#    np.savetxt(FOUT, X_train, delimiter=',')  #only for 2 rd csv file

#for 4d use
#np.save('X_train_split.dat', X_train)
#which can then be loaded using np.load;
#loaded_array = np.load('file_name.npy')


In [0]:
#np.save('X_test_split.dat', X_test)
#np.save('y_train_split.dat', y_train)
#np.save('y_test_split.dat', y_test)

In [0]:
#Build model
model = Sequential()
model.add(Conv2D(filters=16,kernel_size=(5,5), activation='relu',input_shape=(300,300,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu' ))
model.add(Dropout(0.5))
model.add(Dense(25,activation='sigmoid'))

In [0]:
model.summary()

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.save(proj_dir + 'initial_model.h5')

In [0]:
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test),
          callbacks= CSVLogFile, batch_size=64)


#TRAIN THIS ON COLAB AND THEN USE TRAINED MODEL TO PREDICT AS WE DID WITH PYIMAGE VIDEO-CLASS
#use model = keras.models.load_model('path_to_my_model.h5')
model.save(proj_dir + 'trained_model_50epochs.h5')

In [0]:
with open (trainingHistory, 'wb') as f:
  pickle.dump(history.history, f1)

In [0]:
print(model.history.history.keys())

In [0]:
train_loss = model.history.history['loss']
val_loss=model.history.history['val_loss']
train_acc=model.history.history['acc']
val_acc=model.history.history['val_acc']
xc=range(50)



In [0]:
#summarise history for loss
fig=plt.figure(figsize=(5,5))
plt.plot(train_loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()
fig.savefig(proj_dir+ 'loss_50epochs.png')

In [0]:
#summarise history for acc
plt.figure(figsize=(5,5))
plt.plot(train_acc)
plt.plot(val_acc)
plt.title('model acc')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train','val'], loc='best')
plt.show()
fig.savefig(proj_dir + 'acc_50epochs.png')

In [0]:
proj_dir